In [1]:
import time
import sklearn
import numpy as np
import pandas as pd

import sys
sys.path.append("..")
from baggingPU import BaggingClassifierPU

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random

In [2]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    columnwidth = max([len(x) for x in labels]) + 4
    empty_cell = " " * columnwidth
    print("    " + empty_cell, end=' ')
    for label in labels:
        print("%{0}s".format(columnwidth) % 'pred_' + label, end=" ")
    print()

    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % 'true_' + label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            if cell:
                print(cell, end=" ")
        print()

# import data

In [3]:
#df_raw = pd.read_csv('../data/w-dependence.csv')

#df_raw = pd.read_csv('../data/1place-dependence.csv')

df_raw = pd.read_csv('../data/w-related.csv')


df_raw['label'] = df_raw['label'].astype("int")
print(df_raw.label.value_counts())
print('Has null values', df_raw.isnull().values.any())

1    10000
0    10000
Name: label, dtype: int64
Has null values False


In [4]:
df_raw.head(10)

,p1,p2,p3,p4,p5,p6,p7,p8,label
0,0,1,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,1
2,0,1,1,0,0,0,0,0,1
3,0,0,0,1,0,0,0,0,1
4,1,0,1,0,0,0,0,0,1
5,0,0,0,0,1,1,0,0,1
6,0,0,0,0,0,0,1,0,1
7,0,1,2,0,0,0,0,0,1
8,0,0,1,1,0,0,0,0,1
9,0,0,0,0,0,0,0,1,1


In [5]:
print(df_raw.iloc[:10000,:])
df_max = np.amax(df_raw.iloc[:10000,[2,5]])
print(df_max)

      p1  p2  p3  p4  p5  p6  p7  p8  label
0      0   1   0   0   0   0   0   0      1
1      1   0   0   0   0   0   0   0      1
2      0   1   1   0   0   0   0   0      1
3      0   0   0   1   0   0   0   0      1
4      1   0   1   0   0   0   0   0      1
...   ..  ..  ..  ..  ..  ..  ..  ..    ...
9995   0   0   6   0   1  38   0   0      1
9996   0   0   6   0   0  37   1   0      1
9997   1   0   6   0   0  37   0   0      1
9998   0   0   4   0   1  39   0   0      1
9999   0   0   4   0   0  38   1   0      1

[10000 rows x 9 columns]
p3    81
p6    40
dtype: int64


In [6]:
print(df_raw.iloc[10000:,:])
print(np.amax(df_raw.iloc[10000:,[2,5]]))

       p1  p2  p3  p4  p5  p6  p7  p8  label
10000   1   1  46   1   1  34   1   1      0
10001   0   0  17   1   1  22   0   1      0
10002   0   0  53   0   1  32   1   1      0
10003   1   1  55   1   1  10   0   0      0
10004   1   1   6   1   1  20   1   1      0
...    ..  ..  ..  ..  ..  ..  ..  ..    ...
19995   6   0  63   5   7  25   8  10      0
19996   3   6  31   2   9  38  10   3      0
19997   1   3  11   9  10  17   3   9      0
19998  10   9  33   9   7  18   0   2      0
19999   5   3  37   4   2  28   2   1      0

[10000 rows x 9 columns]
p3    81
p6    40
dtype: int64


In [7]:
neg_process = df_raw.iloc[10000:,:].copy()
print(neg_process)

for i in range(10000):
    if neg_process.iloc[i,0] > df_max[0]:
        neg_process.iloc[i,2]  = int(neg_process.iloc[i,2] / df_max[0])
    if neg_process.iloc[i,1] > df_max[1]:
        neg_process.iloc[i,5]  = int(neg_process.iloc[i,5] / df_max[1])

       p1  p2  p3  p4  p5  p6  p7  p8  label
10000   1   1  46   1   1  34   1   1      0
10001   0   0  17   1   1  22   0   1      0
10002   0   0  53   0   1  32   1   1      0
10003   1   1  55   1   1  10   0   0      0
10004   1   1   6   1   1  20   1   1      0
...    ..  ..  ..  ..  ..  ..  ..  ..    ...
19995   6   0  63   5   7  25   8  10      0
19996   3   6  31   2   9  38  10   3      0
19997   1   3  11   9  10  17   3   9      0
19998  10   9  33   9   7  18   0   2      0
19999   5   3  37   4   2  28   2   1      0

[10000 rows x 9 columns]


In [8]:
neg_process.drop_duplicates(inplace=True)
print(neg_process)
print(np.amax(neg_process))

       p1  p2  p3  p4  p5  p6  p7  p8  label
10000   1   1  46   1   1  34   1   1      0
10001   0   0  17   1   1  22   0   1      0
10002   0   0  53   0   1  32   1   1      0
10003   1   1  55   1   1  10   0   0      0
10004   1   1   6   1   1  20   1   1      0
...    ..  ..  ..  ..  ..  ..  ..  ..    ...
19995   6   0  63   5   7  25   8  10      0
19996   3   6  31   2   9  38  10   3      0
19997   1   3  11   9  10  17   3   9      0
19998  10   9  33   9   7  18   0   2      0
19999   5   3  37   4   2  28   2   1      0

[10000 rows x 9 columns]
p1       10
p2       10
p3       81
p4       10
p5       10
p6       40
p7       10
p8       10
label     0
dtype: int64


In [9]:
df_process = pd.concat([df_raw.iloc[:10000,:], neg_process])
print(df_process)

       p1  p2  p3  p4  p5  p6  p7  p8  label
0       0   1   0   0   0   0   0   0      1
1       1   0   0   0   0   0   0   0      1
2       0   1   1   0   0   0   0   0      1
3       0   0   0   1   0   0   0   0      1
4       1   0   1   0   0   0   0   0      1
...    ..  ..  ..  ..  ..  ..  ..  ..    ...
19995   6   0  63   5   7  25   8  10      0
19996   3   6  31   2   9  38  10   3      0
19997   1   3  11   9  10  17   3   9      0
19998  10   9  33   9   7  18   0   2      0
19999   5   3  37   4   2  28   2   1      0

[20000 rows x 9 columns]


In [10]:
x_data = df_process.iloc[:,:-1]
y_data = df_process.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [11]:
x_train_input = pd.concat([x_train.iloc[:,:], y_train],axis=1)
x_test_input = pd.concat([x_test.iloc[:,:],y_test],axis=1)

print(len(x_train_input))
print(len(x_test_input))

print(x_train_input.label.value_counts())
print('Has null values', x_train_input.isnull().values.any())

print(x_test_input.label.value_counts())
print('Has null values', x_test_input.isnull().values.any())

16000
4000
1    8077
0    7923
Name: label, dtype: int64
Has null values False
0    2077
1    1923
Name: label, dtype: int64
Has null values False


In [12]:
x_train_input.head(10)

,p1,p2,p3,p4,p5,p6,p7,p8,label
8118,1,0,16,0,0,28,0,0,1
10171,1,0,33,0,0,36,0,1,0
9399,0,1,4,0,0,37,0,0,1
12669,0,1,9,1,1,32,1,0,0
18809,5,7,70,7,2,31,10,10,0
13869,1,1,12,1,0,17,1,1,0
3320,0,1,14,0,0,16,0,0,1
14689,0,1,55,1,1,28,0,0,0
13087,1,0,71,1,1,33,1,1,0
15992,1,1,62,0,1,2,1,1,0


In [13]:
x_test_input.head(10)

,p1,p2,p3,p4,p5,p6,p7,p8,label
14356,0,1,48,1,1,7,0,0,0
3439,0,0,27,1,0,10,0,0,1
12153,0,0,24,1,1,4,0,1,0
15029,0,1,72,1,0,30,0,0,0
18549,3,5,37,4,10,27,5,1,0
15762,0,0,65,1,1,36,0,1,0
12313,1,1,37,1,1,33,1,1,0
16034,0,0,25,0,1,21,1,1,0
11496,1,0,14,1,1,33,1,0,0
5653,0,0,34,0,1,14,0,0,1


In [14]:
df = x_train_input.copy()

NON_LBL = [c for c in df.columns if c != 'label']
X = df[NON_LBL]
y = df['label']

y_orig = y.copy()

hidden_size = 5000
y.loc[
    np.random.choice(
        y[y == 1].index, 
        replace = False, 
        size = hidden_size
    )
] = 0

print(X)
print(y)

       p1  p2  p3  p4  p5  p6  p7  p8
8118    1   0  16   0   0  28   0   0
10171   1   0  33   0   0  36   0   1
9399    0   1   4   0   0  37   0   0
12669   0   1   9   1   1  32   1   0
18809   5   7  70   7   2  31  10  10
...    ..  ..  ..  ..  ..  ..  ..  ..
13927   0   1   2   0   1  36   0   1
919     0   0  20   1   0   2   0   0
5699    0   0   4   0   0  28   1   0
10742   1   1  37   1   0   7   1   1
16921   1   1  31   1   1  18   1   1

[16000 rows x 8 columns]
8118     0
10171    0
9399     0
12669    0
18809    0
        ..
13927    0
919      1
5699     1
10742    0
16921    0
Name: label, Length: 16000, dtype: int64


In [15]:
pd.Series(y).value_counts()

0    12923
1     3077
Name: label, dtype: int64

# Trainning directly

In [16]:
print('Training XGboost model ...')

#import xgboost as xgb

#model = xgb.XGBClassifier()

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()


#from sklearn.neural_network import MLPClassifier

#model = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(1), random_state=1,max_iter=100000)

model.fit(X, y)

print('Done')

Training XGboost model ...
Done


In [17]:
print('---- {} ----'.format('XGboost model'))
print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test_input.iloc[:,:-1])), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_test, model.predict(x_test_input.iloc[:,:-1])))
print('Recall: ', recall_score(y_test, model.predict(x_test_input.iloc[:,:-1])))
print('Accuracy: ', accuracy_score(y_test, model.predict(x_test_input.iloc[:,:-1])))
print('f1_score: ', f1_score(y_test, model.predict(x_test_input.iloc[:,:-1])))

---- XGboost model ----
                        pred_negative        pred_positive 
           true_negative       2077.0          0.0 
           true_positive       1923.0          0.0 
None

Precision:  0.0
Recall:  0.0
Accuracy:  0.51925
f1_score:  0.0


# Training by bagging

In [18]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()

from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier()

from sklearn.neural_network import MLPClassifier
model3 = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(1), random_state=1,max_iter=100000)

from sklearn import svm
model4 = svm.LinearSVC()

from sklearn.naive_bayes import MultinomialNB
model5 = MultinomialNB()

from sklearn.ensemble import RandomForestClassifier
model6 = RandomForestClassifier(n_estimators = 50,n_jobs = -1)

import xgboost as xgb
model7 = xgb.XGBClassifier()


In [19]:
model_list = [model1, model2, model3, model4, model5, model6, model7]
f1 = np.zeros([len(model_list)],dtype=np.float32)

In [20]:
for i,j in zip(model_list,range(len(model_list))):
    model = BaggingClassifierPU(i,
                         n_estimators = 50, 
                         n_jobs = -1, 
                         max_samples = sum(y)  
                        )
    model.fit(X,y)
    print(f1_score(y_orig, model.predict(X)))
    f1[j] = f1_score(y_orig, model.predict(X))

print(f1)

1.0
0.8951508104780795
1.0
1.0
0.78735310576385
0.9573355709029884
0.9911316512615538
[1.         0.8951508  1.         1.         0.7873531  0.9573356
 0.99113166]


In [21]:
f1_index = []
for i in range(len(f1)):
    if f1[i] >= 0.95:
        f1_index.append(i)
        
print(f1_index)
        
predict_sum = np.zeros([len(X)],dtype=np.float32)
for i in f1_index:
    model = BaggingClassifierPU(model_list[i],
                         n_estimators = 50, 
                         n_jobs = -1, 
                         max_samples = sum(y)  
                        )
    model.fit(X,y)
    predict_sum += model.predict(X)
print(predict_sum)

[0, 2, 3, 5, 6]
[4. 0. 5. ... 5. 0. 0.]


In [22]:
threshold = len(f1_index) / 2
print(predict_sum)

predict = np.zeros([len(y)],dtype=np.int64)

for i in range(len(X)):
    if predict_sum[i] >= threshold :
        predict[i] = 1
    if predict_sum[i] < threshold :
        predict[i] = 0


print(predict)
print(y_orig)

[4. 0. 5. ... 5. 0. 0.]
[1 0 1 ... 1 0 0]
8118     1
10171    0
9399     1
12669    0
18809    0
        ..
13927    0
919      1
5699     1
10742    0
16921    0
Name: label, Length: 16000, dtype: int64


In [23]:
#train data
print('---- {} ----'.format('PU Bagging'))
print(print_cm(sklearn.metrics.confusion_matrix(y_orig,predict), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_orig, predict))
print('Recall: ', recall_score(y_orig,predict))
print('Accuracy: ', accuracy_score(y_orig, predict))
print('f1_score: ', f1_score(y_orig, predict))

---- PU Bagging ----
                        pred_negative        pred_positive 
           true_negative       7923.0          0.0 
           true_positive          0.0       8077.0 
None

Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0


In [24]:
best_model = BaggingClassifierPU(model_list[np.argmax(f1)],
                         n_estimators = 50, 
                         n_jobs = -1, 
                         max_samples = sum(y)  
                        )
print(best_model)

BaggingClassifierPU(base_estimator=LogisticRegression(), max_samples=3077,
                    n_estimators=50, n_jobs=-1)


In [25]:
print('Training bagging classifier...')

pu_start = time.perf_counter()

best_model.fit(X, y)
pu_end = time.perf_counter()
print('Done!')
print('Time:', pu_end - pu_start)


Training bagging classifier...
Done!
Time: 4.321821400983026


In [26]:
#train data
print('---- {} ----'.format('PU Bagging'))
print(print_cm(sklearn.metrics.confusion_matrix(y_orig, best_model.predict(X)), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_orig, best_model.predict(X)))
print('Recall: ', recall_score(y_orig, best_model.predict(X)))
print('Accuracy: ', accuracy_score(y_orig, best_model.predict(X)))
print('f1_score: ', f1_score(y_orig, best_model.predict(X)))

---- PU Bagging ----
                        pred_negative        pred_positive 
           true_negative       7923.0          0.0 
           true_positive          0.0       8077.0 
None

Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0


In [27]:
#print wrong predictions
y_pre = model.predict(X)
y_orig_index = y_orig.index.tolist()

FN_index = []
FT_index = []

for i in range(len(y_orig)):
    if y_orig.iloc[i] == 1 and y_pre[i] == 0 :
        FN_index.append(y_orig_index[i])
    if y_orig.iloc[i] == 0 and y_pre[i] == 1 :
        FT_index.append(y_orig_index[i])
        
print("False Negtive:")
print(df_process.loc[FN_index])
print("False Positive:")
print(df_process.loc[FT_index])

False Negtive:
      p1  p2  p3  p4  p5  p6  p7  p8  label
158    1   0  10   0   0   0   0   0      1
5092   1   0  49   0   0   4   0   0      1
173    0   1  11   0   0   0   0   0      1
24     0   0   3   1   0   0   0   0      1
66     0   0   1   0   1   3   0   0      1
...   ..  ..  ..  ..  ..  ..  ..  ..    ...
809    0   0  22   0   0   0   1   0      1
7479   0   0   9   0   0  30   1   0      1
2491   0   0  35   0   0   2   0   1      1
6229   1   0  31   0   0  16   0   0      1
9877   0   1   6   0   0  37   0   0      1

[144 rows x 9 columns]
False Positive:
Empty DataFrame
Columns: [p1, p2, p3, p4, p5, p6, p7, p8, label]
Index: []


In [28]:
#test data
print('---- {} ----'.format('PU Bagging'))
print(print_cm(sklearn.metrics.confusion_matrix(y_test, best_model.predict(x_test_input.iloc[:,:-1])), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_test, best_model.predict(x_test_input.iloc[:,:-1])))
print('Recall: ', recall_score(y_test, best_model.predict(x_test_input.iloc[:,:-1])))
print('Accuracy: ', accuracy_score(y_test, best_model.predict(x_test_input.iloc[:,:-1])))
print('f1_score: ', f1_score(y_test, best_model.predict(x_test_input.iloc[:,:-1])))

---- PU Bagging ----
                        pred_negative        pred_positive 
           true_negative       2077.0          0.0 
           true_positive          0.0       1923.0 
None

Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0


In [29]:
#print wrong predictions
y_test_pre = best_model.predict(x_test_input.iloc[:,:-1])
y_test_index = y_test.index.tolist()

FN_test_index = []
FT_test_index = []

for i in range(len(y_test)):
    if y_test.iloc[i] == 1 and y_test_pre[i] == 0 :
        FN_test_index.append(y_test_index[i])
    if y_test.iloc[i] == 0 and y_test_pre[i] == 1 :
        FT_test_index.append(y_test_index[i])

print("False Negtive:")
print(df_process.loc[FN_test_index])
print("False Positive:")
print(df_process.loc[FT_test_index])

False Negtive:
Empty DataFrame
Columns: [p1, p2, p3, p4, p5, p6, p7, p8, label]
Index: []
False Positive:
Empty DataFrame
Columns: [p1, p2, p3, p4, p5, p6, p7, p8, label]
Index: []


In [30]:
orig_data = df_process.iloc[:,:-1].copy()
orig_label = best_model.predict(orig_data)

print(orig_data)
print(orig_label)

       p1  p2  p3  p4  p5  p6  p7  p8
0       0   1   0   0   0   0   0   0
1       1   0   0   0   0   0   0   0
2       0   1   1   0   0   0   0   0
3       0   0   0   1   0   0   0   0
4       1   0   1   0   0   0   0   0
...    ..  ..  ..  ..  ..  ..  ..  ..
19995   6   0  63   5   7  25   8  10
19996   3   6  31   2   9  38  10   3
19997   1   3  11   9  10  17   3   9
19998  10   9  33   9   7  18   0   2
19999   5   3  37   4   2  28   2   1

[20000 rows x 8 columns]
[1 1 1 ... 0 0 0]


In [31]:
import xgboost as xgb
model = BaggingClassifierPU(xgb.XGBClassifier(),
                         n_estimators = 50, 
                         n_jobs = -1, 
                         max_samples = sum(y)  
                        )
model.fit(orig_data, orig_label)

BaggingClassifierPU(base_estimator=XGBClassifier(base_score=None, booster=None,
                                                 colsample_bylevel=None,
                                                 colsample_bynode=None,
                                                 colsample_bytree=None,
                                                 gamma=None, gpu_id=None,
                                                 importance_type='gain',
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 n_estimators=1

In [32]:
#verification
import random

place_max = max(np.amax(df_process.iloc[:10000, [2, 5]]))  #w_dependence
name1 = ['min', 'max', 'Precision', 'Recall', 'Accuracy', 'f1_score','time']
test = pd.DataFrame(columns=name1)

for i in range(1, 100):
    pu_start = time.perf_counter()
        
    pos_list = np.random.randint(low=100, high=10000, size=2000).tolist()
    neg_list = np.random.randint(low=10000, high=20000, size=2000).tolist()
    all_list = pos_list + neg_list
    
    flag1 = int((0.1 * (i - 1)) * place_max)
    flag2 = int((0.1 * i) * place_max)
    print(flag2)
    
    temp_data = df_process.iloc[all_list, : -1].copy()
    temp_label = df_process.iloc[all_list, -1].copy()
    
    temp_data.iloc[:, 2] = temp_data.iloc[:, 2] + random.randint((df_max[0] + flag1 + 1), ((df_max[0] + flag2 + 1)))
    temp_data.iloc[:, 5] = temp_data.iloc[:, 5] + random.randint((df_max[1] + flag1 + 1), ((df_max[1] + flag2 + 1)))

    
    print('======')
    print(temp_data.iloc[:, :])

    temp_max = max(np.amax(temp_data.iloc[:, [2, 5]]))
    temp_min = min(np.amin(temp_data.iloc[:, [2, 5]]))

    print('---- {} ----'.format(i))
    print('Precision: ', precision_score(temp_label, model.predict(temp_data)))
    print('Recall: ', recall_score(temp_label, model.predict(temp_data)))
    print('Accuracy: ', accuracy_score(temp_label, model.predict(temp_data)))
    print('f1_score: ', f1_score(temp_label, model.predict(temp_data)))

    orig_data = pd.concat([orig_data, temp_data], ignore_index=True)
    orig_label = pd.Series(orig_label.tolist() + model.predict(temp_data).tolist())
    model.fit(orig_data,orig_label)
    
    pu_end = time.perf_counter()
    
    test.loc[i] = [  temp_min, temp_max,
                        precision_score(temp_label, model.predict(temp_data)),
                        recall_score(temp_label, model.predict(temp_data)),
                        accuracy_score(temp_label, model.predict(temp_data)),
                        f1_score(temp_label, model.predict(temp_data)),
                         (pu_end - pu_start)]
    
test.to_csv('../result/w_related_unknown_result.csv')

8
       p1  p2   p3  p4  p5  p6  p7  p8
1747    0   0   90   0   0  57   0   1
1580    0   0  101   0   0  51   1   0
5338    0   0   88   0   0  71   1   0
3729    0   0  118   0   0  51   0   1
5072    0   1   91   0   0  69   0   0
...    ..  ..  ...  ..  ..  ..  ..  ..
10726   0   1  162   1   1  71   1   1
10045   1   1  128   1   0  60   0   1
13675   1   1  156   1   0  79   1   1
13170   0   1  159   1   0  73   1   1
15785   0   0  114   1   1  67   1   0

[4000 rows x 8 columns]
---- 1 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
16
       p1  p2   p3  p4  p5  p6  p7  p8
2356    1   0  100   0   0  74   0   0
1722    0   0  115   1   0  64   0   0
5043    0   0  117   1   0  75   0   0
9799    0   1  154   0   0  68   0   0
5622    1   0  152   0   0  59   0   0
...    ..  ..  ...  ..  ..  ..  ..  ..
10581   1   1  103   1   0  76   1   1
18729   1  10  127   2   3  68   0   6
13046   1   1   97   1   1  71   0   1
16973   1   1  133   1   1  80   1   1
14

Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
129
       p1  p2   p3  p4  p5   p6  p7  p8
1878    0   0  224   0   1  178   0   0
2911    0   1  217   0   0  185   0   0
9812    0   0  250   1   0  187   0   0
7394    0   0  269   0   1  173   0   0
1813    0   0  233   0   0  172   0   1
...    ..  ..  ...  ..  ..  ...  ..  ..
11511   1   0  249   1   0  183   0   1
11066   1   1  246   1   1  178   1   1
14735   0   1  274   1   1  198   0   1
16383   1   0  275   1   1  178   0   1
14151   0   1  238   1   1  206   1   0

[4000 rows x 8 columns]
---- 16 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
137
       p1  p2   p3  p4  p5   p6  p7  p8
2523    0   0  231   1   0  186   0   0
4800    1   0  232   0   0  193   0   0
1941    0   1  236   0   0  181   0   0
9523    0   1  299   0   0  172   0   0
5586    0   1  239   0   0  192   0   0
...    ..  ..  ...  ..  ..  ...  ..  ..
11731   0   1  274   1   1  191   1   1
17488   1   1  266   0   0  182   0 

Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
243
       p1  p2   p3  p4  p5   p6  p7  p8
8627    0   1  377   0   0  291   0   0
9186    0   0  389   0   1  287   0   0
8186    0   0  367   1   0  295   0   0
2674    0   0  363   0   0  281   1   0
8300    0   0  367   0   0  295   1   0
...    ..  ..  ...  ..  ..  ...  ..  ..
16067   1   1  341   1   1  303   1   1
17117   1   1  399   0   1  300   1   1
19978   4   7  338   0   4  283  10   8
19967   6   5  372   4   3  288  10   3
15436   0   1  356   0   0  303   1   1

[4000 rows x 8 columns]
---- 30 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
251
       p1  p2   p3  p4  p5   p6  p7  p8
2571    0   0  353   0   1  297   0   0
3656    0   0  361   0   0  296   1   0
7290    0   0  394   1   0  290   0   0
705     0   0  331   0   0  297   1   0
531     0   1  347   0   0  288   0   0
...    ..  ..  ...  ..  ..  ...  ..  ..
15246   1   1  376   1   1  301   1   1
19246   5   1  370   5   3  301   7 

Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
356
       p1  p2   p3  p4  p5   p6  p7  p8
352     1   0  450   0   0  398   0   0
9315    0   1  498   0   0  406   0   0
2157    0   0  458   0   0  405   0   1
371     0   1  454   0   0  397   0   0
3970    1   0  462   0   0  410   0   0
...    ..  ..  ...  ..  ..  ...  ..  ..
11183   1   1  502   0   1  414   1   1
12014   1   0  463   1   0  406   1   0
10482   1   0  457   1   0  409   1   0
19432   6   1  495   0   7  427  10   0
15400   1   1  491   1   1  429   1   1

[4000 rows x 8 columns]
---- 44 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
364
       p1  p2   p3  p4  p5   p6  p7  p8
3115    1   0  448   0   0  415   0   0
5524    0   0  498   1   0  398   0   0
1012    0   1  451   0   0  404   0   0
7146    0   0  459   0   0  421   0   1
2369    0   1  475   0   0  399   0   0
...    ..  ..  ...  ..  ..  ...  ..  ..
14140   0   0  499   0   0  401   1   1
12170   1   0  496   1   0  438   0 

Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
469
       p1  p2   p3  p4  p5   p6  p7  p8
7700    1   0  549   0   0  538   0   0
3938    0   0  589   0   1  509   0   0
8372    0   0  613   1   0  508   0   0
1135    1   0  555   0   0  513   0   0
7900    0   0  560   0   0  533   1   0
...    ..  ..  ...  ..  ..  ...  ..  ..
14657   1   1  589   1   1  538   1   1
13816   1   0  581   1   1  526   0   1
11253   1   1  590   1   1  508   0   1
17143   0   1  625   0   1  515   0   0
11562   1   1  616   0   0  537   0   0

[4000 rows x 8 columns]
---- 58 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
477
       p1  p2   p3  p4  p5   p6  p7  p8
8213    0   0  580   1   0  536   0   0
1152    0   0  582   1   0  512   0   0
4640    0   0  561   0   0  536   1   0
6716    0   0  590   0   0  527   0   1
9256    0   0  574   0   0  541   1   0
...    ..  ..  ...  ..  ..  ...  ..  ..
12856   0   0  623   1   1  544   0   0
14971   1   0  619   0   1  541   0 

Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
583
       p1  p2   p3  p4  p5   p6  p7  p8
6140    0   1  685   0   0  639   0   0
7271    0   0  664   0   1  653   0   0
2823    1   0  684   0   0  629   0   0
2688    0   0  687   0   1  628   0   0
8452    0   1  676   0   0  649   0   0
...    ..  ..  ...  ..  ..  ...  ..  ..
16662   1   0  671   0   0  653   0   1
13211   1   0  677   0   1  652   0   1
11643   1   1  690   0   0  625   1   1
18448   5  10  735   5  10  628   5   5
14441   1   1  727   0   1  649   1   1

[4000 rows x 8 columns]
---- 72 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
591
       p1  p2   p3  p4  p5   p6  p7  p8
6578    1   0  726   0   0  630   0   0
1958    0   0  669   0   0  643   0   1
1909    0   0  665   0   0  645   1   0
6741    0   1  685   0   0  651   0   0
3618    0   1  673   0   0  648   0   0
...    ..  ..  ...  ..  ..  ...  ..  ..
15150   1   1  670   1   1  655   1   1
12400   1   0  728   0   1  640   0   0
15539   1   

Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
696
       p1  p2   p3  p4  p5   p6  p7  p8
9410    0   0  846   0   0  738   1   0
4124    0   0  797   0   1  750   0   0
8910    0   0  786   1   0  767   0   0
2196    1   0  805   0   0  738   0   0
8423    0   0  806   1   0  756   0   0
...    ..  ..  ...  ..  ..  ...  ..  ..
11628   1   1  773   0   1  749   0   0
16897   1   1  809   0   1  753   1   1
15344   1   1  847   0   1  760   0   0
11551   1   1  801   1   1  765   1   0
10233   1   1  853   1   1  771   0   0

[4000 rows x 8 columns]
---- 86 ----
Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
704
       p1  p2   p3  p4  p5   p6  p7  p8
6094    0   0  838   0   0  746   0   1
7707    0   1  854   0   0  743   0   0
8553    0   0  806   0   0  768   1   0
8874    0   0  820   1   0  762   0   0
1324    0   0  811   1   0  743   0   0
...    ..  ..  ...  ..  ..  ...  ..  ..
19395   1   2  804   5   9  763  10   6
16481   1   0  832   1   1  778   1   1
17566   1   

Precision:  1.0
Recall:  1.0
Accuracy:  1.0
f1_score:  1.0
